In [98]:
from OpenSSL import crypto
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import os
import time

In [99]:
def normalise_counter(ctr):
    total = sum(ctr.values())
    for key in ctr.keys():
        ctr[key] /=total
    return ctr

In [100]:
benign_df = pd.read_csv('final_certificate_datasets/benign_certs.csv')
benign_urls = pd.read_csv('url_datasets/majestic_million.csv')
phish_df = pd.read_csv('final_certificate_datasets/phish_dataset.csv')
phish_df_urls = pd.read_csv('final_certificate_datasets/phish_dataset_urls.csv')
phish_df

,Unnamed: 0,CN,O,C,businessCategory,serialNumber,juristiction,issuer_CN,issuer_O,expired,notBefore,notAfter,verCode,time_collected
0,0,www.abcnike.shop,NaN,NaN,NaN,NaN,NaN,R3,Let's Encrypt,0.0,b'20210326015631Z',b'20210624015631Z',0,1.620480e+09
1,1,*.bangarvideo.xyz,NaN,NaN,NaN,NaN,NaN,Let's Encrypt Authority X3,Let's Encrypt,1.0,b'20201018134251Z',b'20210116134251Z',10,1.620480e+09
2,2,*.google.com,Google LLC,US,NaN,NaN,NaN,GTS CA 1O1,Google Trust Services,0.0,b'20210316192807Z',b'20210608192806Z',0,1.620480e+09
3,3,*.hostgator.com.br,NaN,NaN,NaN,NaN,NaN,Sectigo RSA Domain Validation Secure Server CA,Sectigo Limited,0.0,b'20200908000000Z',b'20210908235959Z',0,1.620480e+09
4,4,*.mailjet.com,Mailjet SAS,FR,NaN,NaN,NaN,DigiCert SHA2 Secure Server CA,DigiCert Inc,0.0,b'20200302000000Z',b'20210514120000Z',0,1.620480e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4954,368,signaturetn.com,NaN,NaN,NaN,NaN,NaN,"cPanel, Inc. Certification Authority","cPanel, Inc.",0.0,b'20210508000000Z',b'20210806235959Z',0,1.620495e+09
4955,372,cpanel.catalyx.biz,NaN,NaN,NaN,NaN,NaN,R3,Let's Encrypt,0.0,b'20210310192820Z',b'20210608192820Z',0,1.620495e+09
4956,375,freeler.de,NaN,NaN,NaN,NaN,NaN,Starfield Secure Certificate Authority - G2,"Starfield Technologies, Inc.",0.0,b'20200704160310Z',b'20220730080815Z',0,1.620495e+09
4957,377,nwcadvocates.com,NaN,NaN,NaN,NaN,NaN,"cPanel, Inc. Certification Authority","cPanel, Inc.",0.0,b'20210330000000Z',b'20210628235959Z',0,1.620495e+09


In [101]:
benign_df.drop_duplicates(subset=['CN'],inplace=True)
benign_df

,Unnamed: 0,CN,O,C,businessCategory,serialNumber,juristiction,issuer_CN,issuer_O,expired,notBefore,notAfter,verCode,time_collected
0,0,*.google.com,Google LLC,US,NaN,NaN,NaN,GTS CA 1O1,Google Trust Services,False,b'20210413101115Z',b'20210706101114Z',0,1.620138e+09
1,1,*.facebook.com,"Facebook, Inc.",US,NaN,NaN,NaN,DigiCert SHA2 High Assurance Server CA,DigiCert Inc,False,b'20210406000000Z',b'20210703235959Z',0,1.620388e+09
3,3,twitter.com,"Twitter, Inc.",US,NaN,NaN,NaN,DigiCert TLS RSA SHA256 2020 CA1,DigiCert Inc,False,b'20210224000000Z',b'20220222235959Z',0,1.620145e+09
4,4,*.instagram.com,"Facebook, Inc.",US,NaN,NaN,NaN,DigiCert SHA2 High Assurance Server CA,DigiCert Inc,False,b'20210321000000Z',b'20210619235959Z',0,1.620382e+09
5,5,www.linkedin.com,LinkedIn Corporation,US,NaN,NaN,NaN,DigiCert SHA2 Secure Server CA,DigiCert Inc,False,b'20210217000000Z',b'20210816235959Z',0,1.620135e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40975,40975,unitedalliedstates.info,NaN,NaN,NaN,NaN,NaN,"cPanel, Inc. Certification Authority","cPanel, Inc.",False,b'20210331000000Z',b'20210629235959Z',0,1.620386e+09
40976,40976,*.dallasarboretum.org,NaN,NaN,NaN,NaN,NaN,Amazon,Amazon,False,b'20201206000000Z',b'20220104235959Z',0,1.620127e+09
40982,40982,*.servicemax.com,"ServiceMax, Inc.",US,NaN,NaN,NaN,DigiCert TLS RSA SHA256 2020 CA1,DigiCert Inc,False,b'20210104000000Z',b'20220204235959Z',0,1.620135e+09
40987,40987,www.bradshawfoundation.com,NaN,NaN,NaN,NaN,NaN,Sectigo RSA Domain Validation Secure Server CA,Sectigo Limited,False,b'20210128000000Z',b'20220128235959Z',0,1.620383e+09


In [102]:
pCNs = phish_df['CN']
bCNs = benign_df['CN']

In [103]:
len(set(pCNs) - set(bCNs))

4735

In [104]:
set(pCNs) - (set(pCNs) - set(bCNs))

{'*',
 '*.8u.cz',
 '*.accountservergroup.com',
 '*.agilecrm.com',
 '*.alchemer.com',
 '*.appspot.com',
 '*.aruba.it',
 '*.arvixeshared.com',
 '*.azureedge.net',
 '*.azurewebsites.net',
 '*.b-cdn.net',
 '*.banahosting.com',
 '*.bitbucket.io',
 '*.bitly.com',
 '*.bluehost.com',
 '*.bmetrack.com',
 '*.bravenet.com',
 '*.business.site',
 '*.caspio.com',
 '*.cloudaccess.net',
 '*.cloudfront.net',
 '*.cloudwaysapps.com',
 '*.company.site',
 '*.creatorlink.net',
 '*.criteo.com',
 '*.dadapro.com',
 '*.docsend.com',
 '*.domain.com',
 '*.dropbox.com',
 '*.easywp.com',
 '*.ecwid.com',
 '*.emaze.com',
 '*.evernote.com',
 '*.execute-api.us-east-1.amazonaws.com',
 '*.facebook.com',
 '*.free.fr',
 '*.from.sh',
 '*.g.doubleclick.net',
 '*.godaddysites.com',
 '*.google.com',
 '*.google.com.br',
 '*.googlecode.com',
 '*.googleusercontent.com',
 '*.goserver.host',
 '*.gridhost.co.uk',
 '*.gridserver.com',
 '*.gstatic.com',
 '*.hekko.pl',
 '*.herokuapp.com',
 '*.hi.is',
 '*.home.pl',
 '*.hoster.by',
 '*.h

In [45]:
#example of webhosting campaigns on s3

In [36]:
bad_urls = phish_df_urls.loc[phish_df_urls['CN'] == '*.s3.eu-central-1.amazonaws.com']['url']
for u in bad_urls:
    print(u)

http://erqj3aq333jqa3jaowaaauth.s3.eu-central-1.amazonaws.com/index.htm
https://chunliuzo.s3.eu-central-1.amazonaws.com/index.html?email=aaaa@example.jp


In [44]:
clean_df = phish_df.loc[phish_df['CN'].isin(set(pCNs) - set(bCNs))]
clean_df

,Unnamed: 0,CN,O,C,businessCategory,serialNumber,juristiction,issuer_CN,issuer_O,expired,notBefore,notAfter,verCode,time_collected
0,0,www.abcnike.shop,NaN,NaN,NaN,NaN,NaN,R3,Let's Encrypt,0.0,b'20210326015631Z',b'20210624015631Z',0,1.620480e+09
1,1,*.bangarvideo.xyz,NaN,NaN,NaN,NaN,NaN,Let's Encrypt Authority X3,Let's Encrypt,1.0,b'20201018134251Z',b'20210116134251Z',10,1.620480e+09
3,3,*.hostgator.com.br,NaN,NaN,NaN,NaN,NaN,Sectigo RSA Domain Validation Secure Server CA,Sectigo Limited,0.0,b'20200908000000Z',b'20210908235959Z',0,1.620480e+09
5,5,raa.namecheap.com,NaN,NaN,NaN,NaN,NaN,Sectigo RSA Domain Validation Secure Server CA,Sectigo Limited,0.0,b'20201111000000Z',b'20211129235959Z',0,1.620480e+09
6,7,thetoasthost.com,NaN,NaN,NaN,NaN,NaN,"cPanel, Inc. Certification Authority","cPanel, Inc.",0.0,b'20210403000000Z',b'20210702235959Z',0,1.620480e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4954,368,signaturetn.com,NaN,NaN,NaN,NaN,NaN,"cPanel, Inc. Certification Authority","cPanel, Inc.",0.0,b'20210508000000Z',b'20210806235959Z',0,1.620495e+09
4955,372,cpanel.catalyx.biz,NaN,NaN,NaN,NaN,NaN,R3,Let's Encrypt,0.0,b'20210310192820Z',b'20210608192820Z',0,1.620495e+09
4956,375,freeler.de,NaN,NaN,NaN,NaN,NaN,Starfield Secure Certificate Authority - G2,"Starfield Technologies, Inc.",0.0,b'20200704160310Z',b'20220730080815Z',0,1.620495e+09
4957,377,nwcadvocates.com,NaN,NaN,NaN,NaN,NaN,"cPanel, Inc. Certification Authority","cPanel, Inc.",0.0,b'20210330000000Z',b'20210628235959Z',0,1.620495e+09


In [ ]:
clean_df.to_csv('final_certificate_datasets/phish_dataset_nowebhost.csv')

In [97]:
Counter(phish_df_urls['CN']).most_common(200)

[('*.google.com', 1207),
 ('sni.cloudflaressl.com', 867),
 ('*.appspot.com', 485),
 ('*.web-hosting.com', 364),
 ('*.sharepoint.com', 247),
 ('raa.namecheap.com', 246),
 ('firebaseapp.com', 234),
 ('*.weebly.com', 232),
 ('misc-sni.blogspot.com', 202),
 ('*.000webhostapp.com', 194),
 ('*.yolasite.com', 152),
 (nan, 149),
 ('bit.ly', 144),
 ('*.mailjet.com', 117),
 ('*.hostgator.com', 111),
 ('fearless-leader.com', 105),
 ('d.sni.fastly.net', 97),
 ('*.websitewelcome.com', 96),
 ('Plesk', 81),
 ('forms.office.com', 76),
 ('*.webhostbox.net', 71),
 ('*.inmotionhosting.com', 62),
 ('web.app', 62),
 ('rb.gy', 58),
 ('pre-taxes-purshases-reward.ca', 55),
 ('ssl464577.cloudflaressl.com', 53),
 ('*.d1pokn4hef5zk.amplifyapp.com', 51),
 ('*.bluehost.com', 49),
 ('*.s.id', 45),
 ('*.justhost.ru', 45),
 ('*.cloudfront.net', 44),
 ('server.1seodev.com', 44),
 ('*.prod.phx3.secureserver.net', 40),
 ('*.registrar-servers.com', 40),
 ('*.wixsite.com', 38),
 ('ssl733565.cloudflaressl.com', 36),
 ('www

# check vs whole dataset, this is crude because it's not actual cert CNs, but we clearly miss some webhost sites

In [105]:
bURLs = benign_urls['Domain']
len(bURLs)

1000000

In [106]:
len(set(pCNs))

4958

In [107]:
len(set(pCNs)-set(bURLs))

4824

In [108]:
new_CNs = (set(pCNs)-set(bURLs))- set(bCNs)
len(new_CNs)

4652

In [109]:
new_CNs

{'event-ffgratishadiah.duckdns.org',
 'crystaltech.co.ke',
 'wave.bro.kiev.ua',
 'usid.cf',
 'alumdecor.ru',
 'tmphysio.de',
 'theumashow.com',
 'nwob.personal-payee-alert.com',
 'customer-service-client.gcommindia.com',
 '*.sohbetbul.net',
 '*.fra1.digitaloceanspaces.com',
 'cpcontacts.albertinainc.com',
 'buyweighingmachine.com',
 'grupotechbox.com',
 'webdisk.zeebracross.com',
 'autodiscover.thebodinfamily.org',
 'shtuchki.store',
 'cpcontacts.aajicare.in',
 'pay02-olx.info',
 'm.ecure.runescape.com-n.ru',
 'livelobb.ajuda-sms.com',
 'get-payment.irs.gov.covidirs.com',
 '*.savana-restaurant.com',
 'logex.com.tr',
 'packstation-226.com',
 'mashalezhen.com',
 'www.beveiligdicswebserviceencrypt.ml',
 'hrsupportwhm.com',
 'liaisoncas.co',
 '01251000.xyz',
 'sagacontratrading.com',
 'Default SSL Certificate',
 'bingingwithbabish.co',
 'www.bancodeoccidente.didacsis.com',
 'mail.notnot-everyjoincome.duckdns.org',
 'graymontmedical.ga',
 'centroestrategiayliderazgo.com',
 'khmernetworktrai

In [131]:
ctr = Counter(phish_df_urls['CN'])
new_ctr = []
for k in ctr.keys():
    if k in new_CNs:
        new_ctr += [k for _ in range(ctr[k])]

In [132]:
Counter(new_ctr).most_common(30)

[('*.sharepoint.com', 247),
 ('raa.namecheap.com', 246),
 ('*.000webhostapp.com', 194),
 ('fearless-leader.com', 105),
 ('*.websitewelcome.com', 96),
 ('forms.office.com', 76),
 ('pre-taxes-purshases-reward.ca', 55),
 ('ssl464577.cloudflaressl.com', 53),
 ('*.d1pokn4hef5zk.amplifyapp.com', 51),
 ('*.s.id', 45),
 ('server.1seodev.com', 44),
 ('*.registrar-servers.com', 40),
 ('*.wixsite.com', 38),
 ('ssl733565.cloudflaressl.com', 36),
 ('www.amazon-check-co-jp.c5j.top', 35),
 ('*.prodns.com.br', 28),
 ('email302.com', 25),
 ('test-test123.groupe-maillane.fr', 25),
 ('greatmusica.com', 23),
 ('*.hostgator.com.br', 22),
 ('dpssmk.gov.ua', 21),
 ('*.calebetienne.com', 20),
 ('aboydandhisblog.com', 19),
 ('*.htmlcomponentservice.com', 19),
 ('villalarnia.org', 18),
 ('or-frservice.3eclv.com', 17),
 ('*.4everproxy.com', 16),
 ('*.produkdigitalonline.com', 16),
 ('*.afristream.tv', 16),
 ('*.hostgator.mx', 14)]

In [84]:
newnew = []
for pcn in pCNs:
    bust = False
    for u in bURLs:
        if str(pcn).endswith(u):
            print(pcn)
            bust=True
            break
    if not bust:
        newnew.append(pcn)

*.google.com
*.hostgator.com.br
*.mailjet.com
raa.namecheap.com
thetoasthost.com
*.web-hosting.com
fearless-leader.com
ssl733565.cloudflaressl.com
*.hostgator.in
blumen-real.de
*.bluehost.com
mehrozrana.com
*.privatednsorg.com
*.000webhostapp.com
firebaseapp.com
www.physicianbillers.com
srv74.niagahoster.com
www.eposvztd.co.jp.pinjianfang.com
ssl712509.cloudflaressl.com
*.sucuri.net
*.sharepoint.com
*.okitup.net
*.hosting.reg.ru
*.service-now.com
*.webhostbox.net
t.ssl.fastly.net
cpanel10wh.bkk1.cloud.z.com
www.raikuten.co.jp.zuudgaqf.com
*.cloudfront.net
abstractti.com.br
*.aashnamalani.com
hosting.multimedialab.co
*.hostgator.com
wh05.024reg.com
*.websitewelcome.com
s5.create.com
acces-prosperity-limited.com
amzget.com
acanthuspress.co.uk
flourishformums.com
*.weebly.com
intranet.elringklinger.safe-browsing.de
*.prod.sin3.secureserver.net
www.abadiss.com
www.livepuppycam.maximilianschnauzers.com
*.squarespace.com
scalextricman.co.uk
*.94fmsi.com.br
ssl464577.cloudflaressl.com
alumni.

www.shiboar.com
*.cloudwaysapps.com
*.webspaceconfig.de
forms.office.com
www.turismo.inatel.pt
pitbullarms.com
ofckenya.com
bit.do
acquabombas.com.br
cluster029.hosting.ovh.net
*.startlogic.com
*.trendmicro.com
pinjamanpantaitimur.com
backupserver.pentaedro.com
*.easy.co
packstation-221.com
alcohol.aonwardclinic.com
wwwmi3-ss47.a2hosting.com
shikshatutors.org
mapp.18she.com
t.co
amazonseoservices.com
agrocom.net
swiftobjectstorage.eu-frankfurt-1.oraclecloud.com
pikespeakhomesearch.com
ns115.hostinglotus.net
valemercado.com.br
awcp052.cpanel.au.com
wwws.jp-bank.japanpost.jp
abogadomedellin.com.co
segurancabb.norwayeast.cloudapp.azure.com
*.mystrikingly.com
*.wapkiz.com
*.azureedge.net
ssl473630.cloudflaressl.com
*.vo.msecnd.net
arti.ua1.d5.com.ua
mardonesycia.com
myrs.ge
citrinesys.webhostingireland.ie
funkygas.info
joshuablackwilkins.com
forumdecorator.com
*.prod.ams1.secureserver.net
awareness-brusselsairport.be
*.criteo.com
j.mp
mail.voicebanglabd24.com
*.gridhost.co.uk
ssl893663.clo

cliniqtec.com
aibbankings.com
webdisk.charperimagedesign.com
29studios-com02.nh-serv.co.uk
*.hosttech.eu
*.rewardgateway.co.uk
rm-uk-delivery-fee.com
agendabeliving.com.br
ambientaris.com
codesandbox.io
gayatriprojects.com
protect.sabzgoltab.com
www.tsuzuki.co.id
netflix.updatebillingdetail.com
www.t-online.de.rongouniversitychaplaincy.com
www.myauthorz.com
bcpzonasegurasbetas.cndigisol.com
finanziamento-espresso.com
www.wagruopsnetjoin61.wikaba.com
acemlna.com
autodiscover.littleelmapartments.com
ktpn.kalisz.pl
*.r.ag.d.sendibm3.com
peds-ortho.com
autodiscover.megacredi.com
www.firstireland.com
*.tokullarmobilya.com
santoshwomencarehospital.com
webmail.qdevtools.com
mail.glsword.com
*.adscouponaccountscampaign.com
ilovemymess.com
blinusa.com
*.htmlcomponentservice.com
betterbacktogether.com
sikikoparoles.com
*.inforia.net
recso.org
flixpassed.com
alfrescocomforts.com
*.islaspacificos.com
jalapaonews.com.br
velvet.by
biolinky.co
eletronicarwm.com.br
unpaid-transaction.com
webdisk.parki

timeopinion.com
activaciondeprductos.com
www.magix-photos.com
madelynmemoirs.com
www.empresas.netinterbank.interbol-portal.com
cpcalendars.patricepurnell.com
jewellerystore.com.au
*.tmweb.ru
alternatifklinik.com
rbcmontgomery.com
*.ning.com
solarwattafrica.com
urbenorte.com
cpcontacts.clinicavillavitoria.com.br
freevbuckx.com
wz.cz
rabots.org
www.123formbuilder.com
realestatewithsheneka.com
servlces.runescape.com-cn.ru
managehosting.areaclienti.aruba.it.u1336879.cp.regruhosting.ru
e-bay-freelistings-offers-today-2991832.saccgpi.com
mybigfatlistbuilder.com
ronin47design.com
*.topspinllc.com
eneconpanama.net
rioverdepar.com.br
droplr.com
foamnflow.com
*.business.site
*.sslseguridad.com
tubepchiunuoc.com
verify-mynewpayee-support.com
snowleopard-llc.com
*.1blu.de
*.d297c2yt8lktld.amplifyapp.com
cpanel.orcapm.com
appleid.locationinfo.ru
autodiscover.specialwayy.com
moracantik.com
www.pu6gmobile.com
milliondollarsproject.fxfdq.com
alkawaterdiy.com
builderall.com
spotlfy-subscribe.com
gmailv

redredget.com
*.bbcartoes.net
*.at.ua
builderallwp.com
www.tempzter.com
service-hspaymentupdate.com
*.arimanurung.skom.id
*.via.com
drcarsoluciones.com
*.d340w0rs0ab6uq.amplifyapp.com
undertakinggaming.com
jerdanfantasysports.com
webmail.ronaldjamesgroup.co
test.dxbproductions.com
tanias-accounting.co.za
rolasellsrealestate.com
starsoftheindustry.com
proxima-net.com
mudraassociategroups.kitxcart.com
*.azurestaticapps.net
www.pizfirepizzacafe.com
securesquared.com
three-retail-live.devicetradein.co.uk
hotm.art
cpcalendars.indybytes.com
abtekdoor.com
smediaphotography.com
soulhealthlife.com
webdisk.eslickcreative.com
mitrasolusiseragam.com
*.ithemeshosting.com.php73-39.lan3-1.websitetestlink.com
grintechindia.com
www.real-markt.de
*.b-cdn.net
www.imcreator.com
sanpchat.com.ghasalah.com
techdirectbh.bodyworkcloud.com
www.asiiadinova.goosecreektradingpost.com
tudosobretudo.blog.br
variazione-pin.com
talkingdogsmobile.com
meat.uniandes.edu.co
cpcalendars.valenteplay.com.br
webdisk.lindalpil

www.anjelbritt.danielimages.com
dsvgg.com
*.cranetech.com.br
www.asupanngakakterbaru.com
*.getco-genetics.com
5co.com
orange.fr.sanpedroandjankowitz.com
foregon.com
*.cudl.com
newimagedentistryimplants.com
www.coastals24.com
*.smore.com
credilatam.com
imi.org.au
sbcgloballoginn.com
goravia.ru
caixa.apps1.sg-host.com
www.thebeerbuddy.com
olxpay.delivery-c.com
irs.gov.account-robinhood.com
*.staging.wpengine.com
b3276083183900402.temporary.link
*.optica-panorama.com
lisa-privatepa-online.co.uk
www.nuevalyon-cl.nuevalyon.com
digitaltaxmatters.co.uk
jsitor.com
vivaanadventure.com
cpanel.dominioits.com
tinavegaphotography.com
www.niktarpublications.com
*.em.cosmopolitan-lv.com
skribbl-io.org
cluebazar.com
ph.zanqap.com
*.dreamhosters.com
*.one21.in
na.infosec-prod.com
*.reidasplaquinhas.com.br
7yatra.com
test.detkam-vrn.ru
saudiwallcovering.com
*.galiciasegura.com
biocurerx.com
www.banhywkaie.com
flowtork.com
buildingtradesnetwork.com
reschedule-delivery-fee.com
login.microsoftonline.us
cpc

cpcalendars.m-chat-whatsapp69.duckdns.org
frinun.com
*.zxcs.nl
bokep2021-viral.duckdns.org
cpcontacts.eldredgelumber.com
nwdeit.cn
*.gogcdn.net
caixa.siteb.sg-host.com
officeoutlooklog.hopto.org
revistasuninter.com
www.praseco.com
*.wixsite.com
www.orobaba.com
skyecorp.com.au
sagacontratrading.com
*.frimujnry.com
*.magasinaudiolight.com
growingupawesome.com
get-payment.irs.gov.covidirs.com
marceloandre.repl.co
2m-informatica.com
test329df.com
smarttravelerpark.com
citavirtualbncr.com
personallink.my.id
vodafone.pay-balance.com
pjprivatewealth.com.au
cpcontacts.albertinainc.com
metamask-restores.net
*.ingress-comporellon.easywp.com
newmainverdict.com
coredeep.co.za
285milanobet.com
www.domsworld.com
traitpowder.com
account-update-linauc.kmlx3cnkw4ocnks9mxqbe.live
ritamarcus.repl.co
cpcalendars.goamegolf.com
ourdreamcars.com
equatorialguineaonline.com
latefinland.com
cloudedupdated.repl.co
mentoring.beautyforashes.org
acessosemprenovo.servecounterstrike.com
passoffice365.ddns.net
methode

mail.cirb.res.in
yeltbook.com
owaa.duckdns.org
isqlglobal.com
horanataxilacc.com
*.jwebnetworks.com
gscommunityspirit.greenschool.org
my.ts3carb.com.tzbyp.com
proprofs.com
join-grubdewasa18.duckdns.org
maroonsgroupllc.com
link-grubdewasa18.duckdns.org
winvillecollege.com
grupwhatsap-vrl14.duckdns.org
service.co.ip.xdzmbot.cn
lookafteryourchildren.com
gobrokersinc.com
ordershippingerror-mymail.com
tariqalaraimi.com
xlncteam.com
viral-tiktok-2021.duckdns.org
bijouoriental.com
psqaa.com
client-return.thomasmaxwellracing.com.au
autodiscover.bg-groups.com
rmdeliveryupdate.com
cotpak.createchsoft.com
videohots123.duckdns.org
autoriparatori.org
jkhmyft.com
instahelp.alphabittech.co
*.vmenu.jp
zazavoice.com
acessoitauparavoce.com
www.httpswww.chase.com.arabicnikah.com
www.gomsapchinhhang.com
hermes-mydeliveryreschedule.com
oraclemart.com
bouncesensation.com
reschedule-my-mail-track-rm.com
reschedule-alert.me
liaisoncas.co
*.dosaamanahil.com
*.salesmatemail.com
hs-online-security-protection-fcs

cmt-unemploymentohio-gov.com
*.konouzm.com
fashionhandyshop.de
breeze-homes.com
cpcalendars.redelivery-ref017.com
gallery.thegemfox.com
parcelforce-customsfees.com
247contracts365.com
event-ffgratishadiah.duckdns.org
starlingsonlinebanking.com
forms.pabbly.com
get-dot.me
cpanel.grub-budi-xyd.duckdns.org
seeeeoi.duckdns.org
scheduleredelivery-hermes.com
kona.jetos.com
amaozn.rqyjs.com
neuralfluency.com
my-hermes-parcel-redelivery-uk.com
ezybid.net
cpcalendars.whatsaapinvite2.duckdns.org
monsite-orange.fr
goodfuture124.co.za
stemflowerspr.com
devtansfoods.com
hermes-secure-redelivery.com
hermes-gb.net
anoznon.co.ip.fmdmyak.cn
review-bills.com
58e29f7d.duckdns.org
krebbersrealestate.com
webmail.whatsappsjoin.duckdns.org
administraton.ddnsking.com
arnazoon.co.jp.ycaivs.cn
woodmachinedc.com
1suntrustonline.com
login-bos24.com
ssyma.com
s4tiva.com
www.whatsaapinvite5.duckdns.org
olx-pl.o-plata.com
hdr-photo.com
demo.purplealmondconsulting.com
cpcontacts.ohgastronomia.com.br
get-payment.menfb

In [85]:
len(newnew)

1685

In [86]:
newnew

['www.abcnike.shop',
 '*.bangarvideo.xyz',
 'www.amazon-check-co-jp.c5j.top',
 'regency.kiwi',
 'rentyouracc.ru',
 'vrpayment.live.itonicsit.de',
 'sitio2.homs.cl',
 'managmentservice.tk',
 nan,
 'agroteck.mx',
 '*.vendum.ro',
 '100templates.ru',
 'testexp',
 'dominguiscv.es',
 'mihosting.host',
 'abadal.com.mx',
 'Plesk',
 'abumarico.ps',
 'msg.arnazon.gq',
 'access-cloudserver365.de',
 'webslave.ispgateway.de',
 'aabyfnigeria2017.org',
 'localhost',
 'ac-engineering.co.id',
 'localhost.localdomain',
 '*.abyssinia.com.tr',
 'abbiati.ws',
 '5470.in',
 'ads.yourpay.live',
 'bellavalentina.hutska.cz',
 'aberdarecottages.co.ke',
 'testee.ntdll.top',
 'r.mailin.fr',
 'aydincanbayraktar.com.tr',
 'sahlh.org',
 'pkbgt.org',
 'blue103.dnsmisitio.net',
 '8985651.vip',
 'mail.beoganutrition.ie',
 'www.amazccon.co-jp.cc',
 'cpanel-centos-s-1vcpu-1gb-tor1-01',
 'almedan.cc',
 'sweet-sutherland.46-101-115-82.plesk.page',
 'sni-support-required-for-valid-ssl',
 'www.skrejieni.lv',
 'www.cc-jp.cc',


In [87]:
prototype_df = pd.DataFrame()
prototype_df['CN'] = newnew
prototype_df.to_csv('url_datasets/harsh_subset.csv')

In [89]:
len(set(newnew) - set(bCNs))

1662

In [90]:
set(newnew) - set(bCNs)

{'crystaltech.co.ke',
 'usid.cf',
 'alumdecor.ru',
 '*.sohbetbul.net',
 'shtuchki.store',
 'cpcontacts.aajicare.in',
 'pay02-olx.info',
 'm.ecure.runescape.com-n.ru',
 'logex.com.tr',
 'packstation-226.com',
 'www.beveiligdicswebserviceencrypt.ml',
 '01251000.xyz',
 'Default SSL Certificate',
 'bingingwithbabish.co',
 'graymontmedical.ga',
 'Kloxo-MR certificate',
 'regency.kiwi',
 'a1web.in',
 'amazon.nt53c.cn',
 'gbusiness.in',
 'mclaren-so.ml',
 '120u.ru',
 'mymelody.or.jp',
 'paypalupdate.ahlusunna.ps',
 'chevreriepierrealaya.fr',
 'homs.cl',
 'amazon-user.auth.jp-4.top',
 'review79y53-ref82.com',
 'webmail.robocupsingapore.org',
 'againforcreating.fromamazonteam.club',
 'sni-support-required-for-valid-ssl',
 'houseofhealing.in',
 'medianet.org.pk',
 'mpljhngpologneviipsx.ml',
 'packssrl.com.ar',
 'pruefung-kundendaten.de',
 'vegam.co',
 'web5.vefold.is',
 'ambosrmaman.ru',
 'bezpieczna-dostawa.com.pl',
 'conijp.shop',
 'www.amzrzon.xyz',
 'demo.fredatorhostlive.xyz',
 'zepfcenter-

In [93]:
prototype_df = pd.DataFrame()
prototype_df['CN'] = list(set(newnew)- set(bCNs))
prototype_df

,CN
0,crystaltech.co.ke
1,usid.cf
2,alumdecor.ru
3,*.sohbetbul.net
4,shtuchki.store
...,...
1657,ruscctv.ru
1658,moncompteorange.me
1659,asseenontvwebstore.co
1660,lu-shared01.dapanel.net


In [94]:
prototype_df.to_csv('url_datasets/harsher_subset.csv')

In [112]:
dff = pd.read_csv('url_datasets/harsher_subset.csv')

In [113]:
dff

,Unnamed: 0,CN
0,0,crystaltech.co.ke
1,1,usid.cf
2,2,alumdecor.ru
3,3,*.sohbetbul.net
4,4,shtuchki.store
...,...,...
1657,1657,ruscctv.ru
1658,1658,moncompteorange.me
1659,1659,asseenontvwebstore.co
1660,1660,lu-shared01.dapanel.net


In [138]:
ctr = Counter(phish_df_urls['CN'])
new_ctr = []
for k in ctr.keys():
    if k in set(dff['CN']):
        new_ctr += [k for _ in range(ctr[k])]

In [139]:
Counter(new_ctr).most_common(30)

[('pre-taxes-purshases-reward.ca', 55),
 ('www.amazon-check-co-jp.c5j.top', 35),
 ('email302.com', 25),
 ('test-test123.groupe-maillane.fr', 25),
 ('villalarnia.org', 18),
 ('*.afristream.tv', 16),
 ('saisevak.corpcon.co.in', 14),
 ('*.hostgator.co', 13),
 ('member-neteller-com-wallet-account-support-login.malles.in', 12),
 ('securite-client.eu', 11),
 ('sni-support-required-for-valid-ssl', 10),
 ('mpljhngpologneviipsx.ml', 10),
 ('scotia.mx', 9),
 ('infomaps.in', 9),
 ('sl.al', 9),
 ('www-cr-mufg-jp.singsts222n.vip', 9),
 ('*.safirhoteli.hr', 9),
 ('*.sindhtvnews.tv', 9),
 ('*.mobileglobal.ae', 9),
 ('sgp1018.siteground.asia', 8),
 ('*.autothrottle.in', 8),
 ('updateiediately.toamazoncardsafari.buzz', 8),
 ('*.k-b.gr', 8),
 ('com-ro.ru', 8),
 ('amazon.jp-jp.co', 8),
 ('rakuten.shopping-cia2.tokyo', 8),
 ('ehjritiotootfrffr.ml', 8),
 ('amazon.u-jp.co', 8),
 ('amazon.o-jp.co', 8),
 ('amazon.i-jp.co', 8)]

In [130]:
pCNs

0         www.abcnike.shop
1        *.bangarvideo.xyz
2             *.google.com
3       *.hostgator.com.br
4            *.mailjet.com
               ...        
4954       signaturetn.com
4955    cpanel.catalyx.biz
4956            freeler.de
4957      nwcadvocates.com
4958           qwikkar.com
Name: CN, Length: 4959, dtype: object

In [121]:
for k in ctr.keys():
    print(k)

*.arvixeshared.com
*.web-hosting.com
*.google.com
*.sharepoint.com
*.hosting.reg.ru
www.amazon-check-co-jp.c5j.top
host2.eskiz.uz
*.weebly.com
*.prod.phx3.secureserver.net
*.lwspanel.com
8b.io
localhost.localdomain
dar.studio
agapeone.net
vps-private.net
raa.namecheap.com
oneclickitsolution.com
ns100968.ip-147-135-15.us
*.mytrafficmanagement.com
*.wpengine.com
*.000webhostapp.com
*.bluehost.com
vrpayment.live.itonicsit.de
registros4.hiperpromocoespts.com
*.ingress-baronn.easywp.com
sarldataservizi.com
*.mailjet.com
auditmessages.com
mail.abindia.org
*.uk-south-2.nxcli.net
*.mfs.gg
firebaseapp.com
av8uas.co.uk
venuecom.com
absolutegiftware.info
*.timeweb.ru
*.bigrock.com
*.cloudfront.net
*.prod.sin3.secureserver.net
*.mysofthof.com
server1.shambika.com
social-logistics.com
*.webhostbox.net
rsb50.rhostbh.com
cloud-linux107.thaidatahosting.com
*.ct.sendgrid.net
sepatubogor.com
webnode.com
alexnoghreh.com
*.webhostingservices.com
*.execute-api.eu-west-1.amazonaws.com
rb.gy
fearless-leader.

www.algotextil.com.br
55ri.cn
sparkasse.scert-team.com
cluebazar.com
bcqqp.cn
*.s3-ap-southeast-1.amazonaws.com
wallieget.com
ttengineering.co.za
www.fiberlinknetwork.com.br
psservlces.runescape.com-ov.ru
autodiscover.dynastyclinic.ae
www.raikuten.co.jp.nd0h8opk.cc
psservices.runescape.com-oa.ru
test.detkam-vrn.ru
webmail.intrinsic.media
soporteapple.us
nongji.52yjh.top
*.faxitalia.com
irs.gov.account-robinhood.com
spreadshirt.xyz
newimagedentistryimplants.com
msautoteile.eu
www.coastals24.com
pservlces.runescape.com-ov.ru
chuyennghiep.vn
cpcalendars.nelsonjustus.com.br
*.cloudaccess.host
sportsmedicsltd.com
*.promio-mail.com
compliance-central.com
steamscomnunyti.com
restaurant.eatpro.jp
*.10gallon.jp
sardarpatelsevadal.com
buceord.com
digitaltaxmatters.co.uk
saiprofit.com
psservices.runescape.com-fn.ru
*.em.cosmopolitan-lv.com
psservlices.runescape.com-n.ru
aaespanol.org
www.digitalschoolfaridabad.rajwebtechnology.com
gstsuvidhakhammam.in
hsbc.co.uk.secureidv.com
mail.fudenterprises.

amazon.ybtty201.cn
webmail.bellespianoclass.com.sg
webmail.hirleicarlos.com.br
codashop-new449.duckdns.org
click2eat.shop
rakuten.shopping-cia1.cfd
fuad.iainkendari.ac.id
namc.org.np
amaozn.rqyjs.com
polskapoczta.plskontas.casa
xfitpalestre.com
sub4sub.co
*.sohbetbul.net
ibos24-login.com
allesinorde.nu
comfilosn.com
amaozaon.co.jp.cqzxfzk.cn
rakuten.co.jp.rakutennvvip.xyz
incroton.com
smkmanbaulihsan.sch.id
smart-answers.in
arnazoon.co.jp.bx2s9mof8xnjkygs0zuy.ajfnajk.com
brooksosterreich.at
cpcontacts.ohgastronomia.com.br
hdr-photo.com
land2.bornafit.ir
mansae.co.za
www.anjuman-e-burhani.org
ibos-online24.com
*.saisoncard-update.jp
clicbbva.co
krebbersrealestate.com
www.yanaalana.com
administraton.ddnsking.com
login-bos24.com
bitcubetv.cf
artdesigntextile.com
cpcalendars.whatsaapinvite2.duckdns.org
pay02-olx.info
npost.pl-getdelivery.club
x369i.cn
58e29f7d.duckdns.org
grapesitech.com
inpost.moferta.live
ssyma.com
ru.mrt.mn
*.konouzm.com
pontositiacumulados.myddns.me
forms.pabbly.com
ol

In [119]:
dff['CN']

0             crystaltech.co.ke
1                       usid.cf
2                  alumdecor.ru
3               *.sohbetbul.net
4                shtuchki.store
                 ...           
1657                 ruscctv.ru
1658         moncompteorange.me
1659      asseenontvwebstore.co
1660    lu-shared01.dapanel.net
1661    davivienda-clientes.xyz
Name: CN, Length: 1662, dtype: object